In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
import numpy as np
from keras.callbacks import Callback
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

import sys

Using TensorFlow backend.


In [2]:
import preprocess as ppx

In [3]:
# a képek betöltése és előfeldolgozása
image_file_names, data = ppx.csv_load()
image_name_dict, images = ppx.load_images(image_file_names)
train_images, train_data = ppx.data_preprocess(images, data, image_name_dict)

[[[189 180 165]
  [181 172 157]
  [189 180 165]
  ..., 
  [181 176 154]
  [164 159 137]
  [205 200 178]]

 [[197 188 173]
  [198 189 174]
  [190 181 166]
  ..., 
  [179 174 154]
  [198 193 173]
  [207 202 182]]

 [[207 198 183]
  [210 201 186]
  [192 183 168]
  ..., 
  [187 184 165]
  [204 201 182]
  [189 186 167]]

 ..., 
 [[ 99  20  25]
  [ 93  21  24]
  [ 72  11   8]
  ..., 
  [  2  17  10]
  [  5  20  13]
  [  9  24  19]]

 [[107  28  33]
  [ 96  24  27]
  [ 69   8   5]
  ..., 
  [  7  20  13]
  [ 10  23  16]
  [ 14  27  20]]

 [[118  39  44]
  [107  35  38]
  [ 85  24  21]
  ..., 
  [ 41  54  45]
  [ 37  50  43]
  [ 33  46  39]]]
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ..., 
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ..., 
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ..., 
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ..., 
 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ..., 
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ..., 
  [0 0 0]
  [0 0 0]
  [0 0 0]]



In [5]:
imgs = np.array(train_images)

In [6]:
labels = np.array([ls[1] for ls in train_data])



encoder = preprocessing.LabelEncoder()
encoder.fit(labels)


encoded_l = encoder.transform(labels)
print(encoded_l)

labels_onehot = to_categorical(encoded_l)

print(labels_onehot)

[ 3  7  4 13  9  8  2  1 12  5  6  0 11 10]
[[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]]


In [7]:
class TrainingHistory(Callback):
    # Tanulási folyamat elején létrehozunk egy-egy üres listát a kinyerni kívánt metrikák tárolása céljából.
    def on_train_begin(self, logs={}):
        # Hiba mértéke a tanító adatokon.
        self.losses = []
        # Hiba mértéke a validációs adatokon.
        self.valid_losses = []
        # A modell jóságát, pontosságát mérő mutatószám a tanító adatokon. 
        self.accs = []
        # A modell jóságát, pontosságát mérő mutatószám a validációs adatokon. 
        self.valid_accs = []
        # A tanítási fázisok sorszámozása.
        self.epoch = 0
    
    # Minden egyes tanítási fázis végén mentsük el, hogy hogyan teljesít aktuálisan a háló. 
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 1 == 0:
            self.losses.append(logs.get('loss'))
            self.valid_losses.append(logs.get('val_loss'))
            self.accs.append(logs.get('acc'))
            self.valid_accs.append(logs.get('val_acc'))
            self.epoch += 1
            
history = TrainingHistory()

In [8]:
# epoch szám
nb_epoch=100

In [9]:
##########################################################
# error esetére, elvileg nem okoz gondot 'jó' esetben sem
import tensorflow as tf
tf.python.control_flow_ops = tf
##########################################################



# előtanított modell betöltése, a fully-connected rétegek nélkül
base_model = InceptionV3(weights='imagenet', include_top=False)
# az utolsó konvolúciós réteg utána egy global average pooling réteget teszünk, ez rögtön "lapítja" (flatten) a 2D konvolúciót
x = base_model.output
x = GlobalAveragePooling2D()(x)
# ezután hozzáadunk egy előrecsatolt réteget ReLU aktivációs függvénnyel
x = Dense(1024, activation='relu')(x)
# és végül egy kimenete lesz a hálónak - a "binary_crossentropy" költségfüggvénynek erre van szüksége
predictions = Dense(labels_onehot.shape[1], activation='softmax')(x)
# a model létrehozása
model = Model(input=base_model.input, output=predictions)

tf


In [10]:
# két lépésben fogjuk tanítani a hálót
# az első lépésben csak az előrecsatolt rétegeket tanítjuk, a konvolúciós rétegeket befagyasztjuk
for layer in base_model.layers:
    layer.trainable = False
# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel két osztályunk van, ezért bináris keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
labels_onehot.shape[1]

14

In [12]:
model.fit(imgs, labels_onehot, batch_size=8, nb_epoch=10, validation_split=0.2, callbacks=[history])

Train on 11 samples, validate on 3 samples
Epoch 1/10
11/11 [==============================] - 4s - loss: 3.0541 - acc: 0.0000e+00 - val_loss: 2.7648 - val_acc: 0.0000e+00
Epoch 2/10
11/11 [==============================] - 2s - loss: 0.4502 - acc: 0.9091 - val_loss: 3.7799 - val_acc: 0.0000e+00
Epoch 3/10
11/11 [==============================] - 2s - loss: 0.0587 - acc: 1.0000 - val_loss: 4.9508 - val_acc: 0.0000e+00
Epoch 4/10
11/11 [==============================] - 2s - loss: 0.0175 - acc: 1.0000 - val_loss: 5.4750 - val_acc: 0.0000e+00
Epoch 5/10
11/11 [==============================] - 2s - loss: 0.2147 - acc: 0.9091 - val_loss: 5.7674 - val_acc: 0.0000e+00
Epoch 6/10
11/11 [==============================] - 2s - loss: 3.0597e-04 - acc: 1.0000 - val_loss: 5.8782 - val_acc: 0.0000e+00
Epoch 7/10
11/11 [==============================] - 2s - loss: 9.7379e-04 - acc: 1.0000 - val_loss: 5.9492 - val_acc: 0.0000e+00
Epoch 8/10
11/11 [==============================] - 2s - loss: 0.0218 

In [13]:
# ehhez először nézzük meg a háló felépítését
print("Az Inception V3 konvolúciós rétegei:")
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# majd a hálónak csak az első 172 rétegét fagyasztjuk, a többit pedig engedjük tanulni
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

Az Inception V3 konvolúciós rétegei:
0 input_1
1 convolution2d_1
2 batchnormalization_1
3 convolution2d_2
4 batchnormalization_2
5 convolution2d_3
6 batchnormalization_3
7 maxpooling2d_1
8 convolution2d_4
9 batchnormalization_4
10 convolution2d_5
11 batchnormalization_5
12 maxpooling2d_2
13 convolution2d_9
14 batchnormalization_9
15 convolution2d_7
16 convolution2d_10
17 batchnormalization_7
18 batchnormalization_10
19 averagepooling2d_1
20 convolution2d_6
21 convolution2d_8
22 convolution2d_11
23 convolution2d_12
24 batchnormalization_6
25 batchnormalization_8
26 batchnormalization_11
27 batchnormalization_12
28 mixed0
29 convolution2d_16
30 batchnormalization_16
31 convolution2d_14
32 convolution2d_17
33 batchnormalization_14
34 batchnormalization_17
35 averagepooling2d_2
36 convolution2d_13
37 convolution2d_15
38 convolution2d_18
39 convolution2d_19
40 batchnormalization_13
41 batchnormalization_15
42 batchnormalization_18
43 batchnormalization_19
44 mixed1
45 convolution2d_23
46 ba

In [14]:
# ez után újra le kell fordítanunk a hálót, hogy most már az Inception V3 felsőbb rétegei tanuljanak
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# és ismét indítunk egy tanítást, ezúttal nem csak az előrecsatolt rétegek,
# hanem az Inception V3 felső rétegei is tovább tanulnak
model.fit(np.asarray(train_images), labels_onehot,  batch_size=16, nb_epoch=10, validation_split=0.2, callbacks=[history])
print("Tanítás vége.")

Train on 11 samples, validate on 3 samples
Epoch 1/10
11/11 [==============================] - 4s - loss: 5.1532e-06 - acc: 1.0000 - val_loss: 7.1752 - val_acc: 0.0000e+00
Epoch 2/10
11/11 [==============================] - 2s - loss: 5.1694e-06 - acc: 1.0000 - val_loss: 7.4286 - val_acc: 0.0000e+00
Epoch 3/10
11/11 [==============================] - 3s - loss: 5.1640e-06 - acc: 1.0000 - val_loss: 7.6748 - val_acc: 0.0000e+00
Epoch 4/10
11/11 [==============================] - 2s - loss: 5.1478e-06 - acc: 1.0000 - val_loss: 7.8993 - val_acc: 0.0000e+00
Epoch 5/10
11/11 [==============================] - 2s - loss: 5.1478e-06 - acc: 1.0000 - val_loss: 8.1275 - val_acc: 0.0000e+00
Epoch 6/10
11/11 [==============================] - 3s - loss: 5.1532e-06 - acc: 1.0000 - val_loss: 8.3686 - val_acc: 0.0000e+00
Epoch 7/10
11/11 [==============================] - 2s - loss: 5.1640e-06 - acc: 1.0000 - val_loss: 8.6247 - val_acc: 0.0000e+00
Epoch 8/10
11/11 [==============================] - 2s

In [15]:
predicted_values = model.predict(np.asarray(train_images))

In [19]:
encoder.inverse_transform(max_pred).shape

(14, 14)

In [18]:
max_pred = np.zeros(predicted_values.shape, dtype=int)
for i, pred in enumerate(predicted_values):
    d = np.argmax(pred)
    max_pred[i, d] = 1
print()

In [ ]:
ecovered_X = np.array([ohc.active_features_[col] for col in out.sorted_indices().indices])
                .reshape(n_samples, n_features) - ohc.feature_indices_[:-1]

In [50]:
np.zeros((3,4), dtype=int)

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

In [39]:
xx = np.array([4,56,67.3,7])

In [59]:
predicted_values.shape

(770, 459)